In [98]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
path_to_insults = '/Users/dattasainathd/Downloads/'
data2 = pd.read_csv(path_to_insults + 'final_train_nonbot.csv',na_values='0', encoding = "ISO-8859-1", nrows = 1477)
data2['Value'] = 0
data2['user'] = 1

In [99]:
data2.head(2)

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listedcount,created_at,...,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot,Value,user
0,34716038.0,34716038,aaroncarter,Sony Recording Artist,NEW AARON CARTER MUSIC DOWNLOAD LINK LÃÃ¼VÃÃ...,NaN,571310.0,76070.0,4909.0,Thu Apr 23 20:15:49 +0000 2009,...,56077.0,en,"{u'contributors': None, u'truncated': False, u...",False,False,True,Aaron Carter,NaN,0,1
1,3013511.0,3013511,michellebranch,NaN,"singer/songwriter, hopeless romantic, drinker ...",https://t.co/2vSKw7cmzC,292385.0,963.0,6076.0,Fri Mar 30 20:58:59 +0000 2007,...,16688.0,en,"{u'contributors': None, u'truncated': False, u...",False,False,False,Michelle Branch,NaN,0,1


In [100]:
data2 = data2.drop('created_at', axis=1)
data2.verified=data2.verified.astype(int)
data2.default_profile=data2.default_profile.astype(int)
data2.default_profile_image=data2.default_profile_image.astype(int)

data2 = data2.drop('status', axis=1)
data2 = data2.drop('id', axis=1)
data2 = data2.drop('id_str', axis=1)
data2 = data2.drop('url', axis=1)
#data2 = data2[data2.verified != 1]
data2 = data2.drop('has_extended_profile', axis=1)

data2.location.fillna('', inplace=True)
data2.description.fillna('', inplace=True)
data2.followers_count.fillna(0, inplace=True)
data2.friends_count.fillna(0, inplace=True)
data2.listedcount.fillna(0, inplace=True)
data2.statuses_count.fillna(0, inplace=True)
data2.favourites_count.fillna(0, inplace=True)
data2.followers_count=data2.followers_count.astype(int)

print("Did all the necessary changes for the data for manipulation")

Did all the necessary changes for the data for manipulation


In [101]:
data_description = data2['description'].str.lower()
data_name = data2['name'].str.lower()
data_screen_name = data2['screen_name'].str.lower()
bot = 'bot'
count = 0
serial = -1
for x in data_screen_name:
    serial = serial + 1
    if bot in x:
        data2.ix[serial, 'Value'] = 2
        count += 1
serial = -1

for x in data_name:
    serial = serial + 1
    try:
        if bot in x:
            data2.ix[serial, 'Value'] = 2
            count += 1
    except:
        pass

serial = -1

for x in data_description:
    serial = serial + 1
    try:
        if bot in x:
            data2.ix[serial, 'Value'] = 2
            count += 1
    except:
        pass

print("trying to search for 'bot' in names, screen_names & description")
print("if existed, we shall change value to 2")
print(count)


trying to search for 'bot' in names, screen_names & description
if existed, we shall change value to 2
38


In [102]:
#trying to remove accounts with zero followers/zero friends/zero tweets
#Bias obtained from given data
serial = -1
count = 0
unnec = []
for x in range(0, 1477):
    try:
        serial = serial + 1
        if(1):
            if(data2.loc[serial, 'statuses_count']==0 or data2.loc[serial, 'followers_count']==0 or 
               data2.loc[serial, 'friends_count']==0 ):
                data2.loc[serial, 'Value'] = 3
                count+=1
    except:
        pass
print("trying to search for accounts with followers, friends or tweets in 0")
print("if there is some accounts like that we will consider them as 'starter' humans and assign value 3")
print(count)

trying to search for accounts with followers, friends or tweets in 0
if there is some accounts like that we will consider them as 'starter' humans and assign value 3
26


In [103]:
#trying to filter accounts usually used by comman man (not celebrities or verified users)
serial = -1
count = 0
unnec = []
for x in range(0, 1477):
    try:
        serial = serial + 1
        if(data2.loc[serial, 'Value']==0):
            if((data2.loc[serial, 'statuses_count']/data2.loc[serial, 'followers_count']<12 
                or data2.loc[serial, 'statuses_count']/data2.loc[serial, 'friends_count']<12)):
                data2.loc[serial, 'Value'] = 3
                count+= 1
    except:
        pass
print("trying to filter accounts usually used by comman man (not celebrities or verified users)")
print("comman users have low tweet/following or tweet/friends and assigned value 3")
print(count)

trying to filter accounts usually used by comman man (not celebrities or verified users)
comman users have low tweet/following or tweet/friends and assigned value 3
1325


In [104]:
#trying to filter accounts which are verified all the verified user accounts are humans
#because twitter verifies users only 
serial = -1
count = 0
unnec = []
for x in range(0, 1477):
    try:
        serial = serial + 1
        if(data2.loc[serial, 'verified']==1):
            data2.loc[serial, 'Value'] = 3
            count+= 1
    except:
        pass
print("trying to filter accounts which are verified all the verified user accounts are humans")
print("twitter user accounts which are verified are assigned to value 3")
print(count)

trying to filter accounts which are verified all the verified user accounts are humans
twitter user accounts which are verified are assigned to value 3
645


In [105]:
#there might be some clever bots who dont have any description but follow huge number of poeople or they are followed
#by huge number.
serial = -1
count = 0
unnec = []
for x in range(0, 1477):
    try:
        serial = serial + 1
        if(data2.loc[serial, 'Value']==1 or data2.loc[serial, 'Value']==0):
            if((data2.loc[serial, 'statuses_count']/data2.loc[serial, 'followers_count']>10 
                or data2.loc[serial, 'statuses_count']/data2.loc[serial, 'friends_count']>10)):
                data2.loc[serial, 'Value'] = 2
                count+= 1
                print(serial)
                print(data2.loc[serial, 'statuses_count']/data2.loc[serial, 'followers_count'])
                print(data2.loc[serial, 'statuses_count']/data2.loc[serial, 'friends_count'])
                
    except:
        pass
print("huge tweeting accounts")
print("huge rate tweet accounts are considered bots and retained values 2")
print(count)

4
39.1517027864
56.7085201794
7
23.1930379747
38.171875
24
22.85
17.9215686275
40
40.1592920354
16.2071428571
49
14.8350515464
13.7047619048
56
28.1592356688
15.7330960854
67
120.401129944
179.084033613
104
16.9830508475
14.9552238806
105
21.7962962963
13.2247191011
221
64.7045454545
118.625
246
23.5789473684
149.333333333
247
40.1226912929
46.503058104
261
21.1255605381
23.0554649266
268
47.1403508772
29.9665427509
279
132.239130435
506.916666667
310
53.7591660676
156.769392034
330
33.4285714286
21.2727272727
331
40.037037037
40.5375
338
55.4824561404
49.4140625
392
36.1565217391
27.0
397
122.233333333
84.7861271676
410
148.615384615
169.05
411
14.0415400203
56.7991803279
435
59.8709677419
47.5897435897
444
22.2918424754
15.276626506
467
64.4407894737
40.643153527
470
24.4248704663
17.5895522388
487
25.3585526316
19.8174807198
492
15.4437320736
92.36629667
505
143.267241379
139.655462185
532
14.1369426752
16.3198529412
548
67.6877470356
133.26848249
557
25.0573870161
26.3806052416
559

In [106]:
data2

,screen_name,location,description,followers_count,friends_count,listedcount,favourites_count,verified,statuses_count,lang,default_profile,default_profile_image,name,bot,Value,user
0,aaroncarter,Sony Recording Artist,NEW AARON CARTER MUSIC DOWNLOAD LINK LÃÃ¼VÃÃ...,571310,76070.0,4909.0,37437.0,1,56077.0,en,0,0,Aaron Carter,NaN,3,1
1,michellebranch,,"singer/songwriter, hopeless romantic, drinker ...",292385,963.0,6076.0,1248.0,1,16688.0,en,0,0,Michelle Branch,NaN,3,1
2,stronginmyfaith,,,1,7.0,0.0,0.0,0,3.0,en,1,1,laurie lindeman,NaN,3,1
3,Jessicaveronica,5th Dimension,"@TheVeronicas â°ÃÂ Wanderer, worshiper, if ...",222659,352.0,3261.0,143.0,1,10999.0,en,0,0,Jessica Veronica,NaN,3,1
4,resargento,brasil,meio ogra mas o coraÃ­_Ã­Â£o Ã­Â© de mocinha,646,446.0,2.0,13440.0,0,25292.0,pt,0,0,rebosta,NaN,2,1
5,crazyl1f,,,2,5.0,0.0,0.0,0,2.0,zh-cn,1,1,Songgaoyun Deng,NaN,3,1
6,TaniasimonTania,Milano,"Mechanical engineer, F1 fan and traveler _ÃÃÂ",27,25.0,0.0,17.0,0,50.0,it,1,0,Tania Simon,NaN,3,1
7,YCPRProf,Ã«ÃÃ«Â£Ã«_-â°_Ãâ¢Ã¼Â-Gamma Iota Advisor,"We are the music makers, and we are the dreame...",316,192.0,9.0,2257.0,0,7329.0,en,0,0,Dr. K. McBride,NaN,2,1
8,bandofhorses,,'Why Are You OK' out now - https://t.co/G7QeyE...,211616,8617.0,4071.0,1941.0,1,2037.0,en,0,0,Band of Horses,NaN,3,1
9,sparker,SF / LA / NYC,"Napster, Plaxo, Facebook, Causes, Spotify & Ai...",429604,600.0,4999.0,30.0,1,497.0,en,0,0,Sean Parker,NaN,3,1


In [107]:
serial = -1
count = 0
unnec = []
for x in range(0, 1477):
    try:
        serial = serial + 1
        if(data2.loc[serial, 'Value']==2):
            data2.loc[serial, 'Value'] = 0
        else:
            data2.loc[serial, 'Value'] = 1
        count+= 1
    except:
        pass
count

1476

In [108]:
data_value = data2['Value']
datafinal = data2['user']
print("Accuracy Score: ", (accuracy_score(datafinal, data_value)))
cnf_matrix = confusion_matrix(datafinal, data_value)
print("Confusion Matrix:")
print(cnf_matrix)

Accuracy Score:  0.917344173442
Confusion Matrix:
[[   0    0]
 [ 122 1354]]


In [109]:
TP = 0
FP = 0
TN = 0
FN = 0

for i in range(len(data_value)): 
    if datafinal[i]==data_value[i]==1:
        TP += 1
        #print(i)
for i in range(len(data_value)): 
    if data_value[i]==1 and datafinal[i]!=data_value[i]:
        FP += 1
        #print(i)
for i in range(len(data_value)): 
    if datafinal[i]==data_value[i]==0:
        TN += 1
        #print(i)
for i in range(len(data_value)): 
    if data_value[i]==0 and datafinal[i]!=data_value[i]:
        FN += 1
        #print(i)